# Tensorflow Sessions statically run Tensorflow Graphs.

If you have some Tensorflow Graph, whether you downloaded or created it from scratch using the tensorflow’s **Graph API**. You will need to use the **Session API** to run it.

We will take the following graph as a simple example. When we call `sess.run()` on `tf.Tensor` types.

`g1.get_operation_by_name` will simply return the `tf.Operation` object. We can get an array of the `tf.Tensor` objects that this operation produces with the `.outputs` attribute. **Remember, it is an important distinction to make that `tf.Tensor` and `tf.Operation` are different**! `tf.Operation` is a node `tf.Tensor` is an edge.



In [1]:
import tensorflow as tf

g1 = tf.Graph()

with g1.as_default():
    my_input = tf.constant([-1,0,1], dtype=tf.float16, name="input")
    a = tf.square(my_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")
    
sess = tf.Session(graph=g1);
print("A:{}".format(sess.run(g1.get_operation_by_name("A").outputs)))
print("B:{}".format(sess.run(g1.get_operation_by_name("B").outputs)))
print("C:{}".format(sess.run(g1.get_operation_by_name("C").outputs)))
print("D:{}".format(sess.run(g1.get_operation_by_name("D").outputs)))
print("E:{}".format(sess.run(g1.get_operation_by_name("E").outputs)))
print("F:{}".format(sess.run(g1.get_operation_by_name("F").outputs)))

A:[array([1., 0., 1.], dtype=float16)]
B:[array([0.5405, 1.    , 0.5405], dtype=float16)]
C:[array([0.8413, 0.    , 0.8413], dtype=float16)]
D:[array([1.382, 1.   , 1.382], dtype=float16)]
E:[array([0., 1., 0.], dtype=float16)]
F:[array([1.176, 1.   , 1.176], dtype=float16)]


The way we run the graph above is actually very inefficient. It is running 6 times. You can see this by [running a quick `tf.Print`](https://gist.github.com/Ouwen/66326b796311f0cca3c60067b9237124). `tf.Print` is a graph node that outputs values and messages to `stderr` from the Tensorflow C++ Runtime.

<img src="../figures/yikes.png" width="400"/>
*Yikes*

We only want to run the graph once and retrieve all outputs. We can easily tweak our code so that we only make one `run` call. We will also notice that the graph is run just once instead of 6 times.

In [2]:
!python tensorflow_graph_example_min_1.py

2018-10-09 08:56:25.770741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla V100-SXM2-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:06:00.0
totalMemory: 31.74GiB freeMemory: 1.15GiB
2018-10-09 08:56:26.582740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 1 with properties: 
name: Tesla V100-SXM2-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:07:00.0
totalMemory: 31.74GiB freeMemory: 1.15GiB
2018-10-09 08:56:26.582855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0, 1
2018-10-09 08:56:27.921101: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-10-09 08:56:27.921139: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 1 
2018-10-09 08:56:27.921165: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N Y 
2018-10-09 08:56:27.921171: I tensor

In [3]:
import tensorflow as tf
from src import cloud_visualizer

g1 = tf.Graph()

with g1.as_default():
    my_input = tf.constant([-1,0,1], dtype=tf.float16, name="input")
    my_printed_input = tf.Print(my_input, [my_input], message="Running the graph", name="print")
    a = tf.square(my_printed_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")
    
sess = tf.Session(graph=g1);

results = sess.run({
    "A": g1.get_operation_by_name("A").outputs,
    "B": g1.get_operation_by_name("B").outputs,
    "C": g1.get_operation_by_name("C").outputs,
    "D": g1.get_operation_by_name("D").outputs,
    "E": g1.get_operation_by_name("E").outputs,
    "F": g1.get_operation_by_name("F").outputs
})
for k, v in sorted(results.items()):
    print(k, v)
cloud_visualizer.show_graph(g1)

A [array([1., 0., 1.], dtype=float16)]
B [array([0.5405, 1.    , 0.5405], dtype=float16)]
C [array([0.8413, 0.    , 0.8413], dtype=float16)]
D [array([1.382, 1.   , 1.382], dtype=float16)]
E [array([0., 1., 0.], dtype=float16)]
F [array([1.176, 1.   , 1.176], dtype=float16)]


We may want to add some more dynamic elements to our graph. This can be done with `tf.Variable` operations and `tf.Placeholder` operations. 

`tf.Placeholder` is a simple operation that takes in a value during the session runtime. Rather than having `my_input` be a constant we can instead use a placeholder.

`tf.Variable` is a bit more interesting… We will just create a graph with one variable to our graph.

In [4]:
import tensorflow as tf
from src import cloud_visualizer

g1 = tf.Graph()
with g1.as_default():
    # Add a single variable to our graph
    v = tf.get_variable(name="v", shape=(), initializer=tf.glorot_uniform_initializer())

sess = tf.Session(graph=g1)
sess.run(v.initializer) # Run just the initializer on our variable

# sess.run(tf.global_variables_initializer()) # This will initialize all variables
print(sess.run(v))
cloud_visualizer.show_graph(g1)

-0.7490073


When we inspect what is added to our graph, we notice that `tf.Variable` is actually a group of many different operations: `tf.Identity`, `tf.Assign`, `tf.VariableV2` and more operations within the the `Initializer`. **These exist to help `tf.Variable` store state.**

When the variable is first willed into existence, it has no value. This is why when you begin a session you must first initialize your variables, and why there is an `Initializer` is attached to the `tf.Variable` class (in our example we use a random distribution. The snippet above can be run and `sess.run(v)` will print out a random number. However, try placing `sess.run(v)` before the variable is init, and you will receive an error.

When using the python API to work with variables, there is under the hood syntax sugar that exists so that you are able to use the variables as though they are normal tensor outputs from a normal operation. A big difference to note is that when we save the graph as a protobuf, the `tf.Variable` group is saved; however the value stored in the `tf.Variable` is lost. In order to save this we must utilize `tf.train.Saver`.

In [5]:
import tensorflow as tf

g1 = tf.Graph()

with g1.as_default():   
    v = tf.get_variable(name="v", shape=(), initializer=tf.glorot_uniform_initializer())
    saver = tf.train.Saver()
    
sess = tf.Session(graph=g1);
print(sess.run(v.initializer))
print(sess.run(v.assign_add(1)))
print(sess.run(v.assign_add(1)))
print(sess.run(v.assign_add(1)))
print(sess.run(v.assign_add(1)))
saver.save(sess, "/tmp/model.ckpt")

None
-0.28589177
0.7141082
1.7141082
2.7141082


'/tmp/model.ckpt'

*We perform some adds to our variable, then we run the saver. **Note** the files produced are actually checkpoint, model.ckpt.data-…, model.ckpt.index, and model.ckpt.meta. When we call /tmp/model.ckpt later, these files will need to exist so don’t move them around.*

In [6]:
import tensorflow as tf

g1 = tf.Graph()

with g1.as_default():   
    v = tf.get_variable(name="v", shape=(), initializer=tf.glorot_uniform_initializer())
    saver = tf.train.Saver()
    
sess = tf.Session(graph=g1);
saver.restore(sess, "/tmp/model.ckpt")
sess.run(v)

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


2.7141082

*try save as protobuf and then read it*

In [7]:
import tensorflow as tf
from google.protobuf import text_format

#write to pbtxt file
tf.train.write_graph(graph_or_graph_def=g1, 
                     logdir='/tmp/storage/', 
                     name='graph_protobuf.pbtxt')

# Let's read our pbtxt file into a Graph protobuf
f = open("/tmp/storage/graph_protobuf.pbtxt", "r")
graph_protobuf = text_format.Parse(f.read(), tf.GraphDef())

# Import the graph protobuf into our new graph.
graph_clone = tf.Graph()
with graph_clone.as_default():
    tf.import_graph_def(graph_def=graph_protobuf, name="")

# Display the graph inline.
graph_clone.as_graph_def()
sess = tf.Session(graph=graph_clone)
# print(sess.run(v)) #this line will fail

*We can run the saver to restore our session variable. Since the value is coming from our checkpoint file, no init is needed.*

To put most simply, the `.ckpt` file is just a map of our variable names to the value that was stored during the session. If I created an entirely new graph, as long as I had the variable name `v` in my graph I would be able to restore the value in the checkpoint.

In production these `.ckpt` files can be run multiple times to snapshot the progress of a Tensorflow graph that is running in a session.

I hope this was a helpful short overview of the low level tensorflow API.

In [8]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='', all_tensors=True)

tensor_name:  v
2.7141082
